In [31]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Functions for creating a dog

In [2]:
WALKING_TIME = 15
EATING_TIME = 1
PLAYING_TIME = 4

food_consumption_rate = 1.0 / (30 * 3600)
affection_consumption_rate = 1.0 / (50 * 3600)
walking_fat_converge_rate = 0.2
walking_affection_converge_rate = 0.4
playing_fat_converge_rate = 0.1
playing_affection_converge_rate = 0.20
eating_food_increase = 0.6
eating_fat_increase = 0.25

In [3]:
def transform_to_dec(value):
    return np.floor(100*value)/100

def apply_decreasing_rate(value: float, rate: float) -> float:
    """
    Apply a decreasing rate to a value
    :param value: current value
    :param rate: per second
    :return: updated value
    """
    return value - (60 * rate)

def converge(value: float, target: float, ratio: float) -> float:
    diff: float = (target - value) * ratio
    return value + diff

def update_walking(dog_table_row):
    update_fat = converge(dog_table_row.iloc[0]['fat'], 0.0, walking_fat_converge_rate)
    update_affection = converge(dog_table_row.iloc[0]['affection'], 1.0, walking_affection_converge_rate)
    return (update_fat, update_affection)

def update_feeding(dog_table_row):
    update_food = min(dog_table_row.iloc[0]['food'] + eating_food_increase, 1.0)
    update_fat = min(dog_table_row.iloc[0]['fat'] + eating_fat_increase, 1.0)
    return (update_food, update_fat)

def update_playing(dog_table_row):
    update_fat = converge(dog_table_row.iloc[0]['fat'], 0.0, playing_fat_converge_rate)
    update_affection = converge(dog_table_row.iloc[0]['affection'], 1.0, playing_affection_converge_rate)
    return (update_fat, update_affection)

def update_food(dog_table_row):
    return transform_to_dec(max(
        0.0,
        apply_decreasing_rate(dog_table_row.iloc[0]['food'], food_consumption_rate)
    ))

def update_affection(dog_table_row):
    return transform_to_dec(max(
        0.0,
        apply_decreasing_rate(dog_table_row.iloc[0]['affection'], affection_consumption_rate)
    ))

def update_happiness(dog_table_row) -> float:  # between 0 and 1
    return min(
        dog_table_row.iloc[0]['food'],
        1.0 - dog_table_row.iloc[0]['fat'],
        dog_table_row.iloc[0]['affection']
    )

def update_alive(dog_table_row):
    return dog_table_row.iloc[0]['alive'] & (dog_table_row.iloc[0]['happiness'] > 0.0)

def new_state_after_1_minute(dog_table, action_taken='NO ACTION'):
    
    dog_table_row = dog_table.tail(1)
    new_state_dog_id = dog_table_row.iloc[0]['dog_id']

    new_state_fat = dog_table_row.iloc[0]['fat']
    new_state_last_action_taken = dog_table_row.iloc[0]['last_action_taken']
    new_state_can_action_be_taken = dog_table_row.iloc[0]['can_action_be_taken']
    new_state_minutes_since_last_action = dog_table_row.iloc[0]['minutes_since_last_action']
    
    new_state_minute = dog_table_row.iloc[0]['minute'] + 1
    new_state_minutes_since_last_action = dog_table_row.iloc[0]['minutes_since_last_action'] + 1
    
    new_state_food = update_food(dog_table_row)
    new_state_affection = update_affection(dog_table_row)
    new_state_happiness = update_happiness(dog_table_row)
    new_state_alive = update_alive(dog_table_row)
    
    if ((action_taken != 'NO ACTION') & (dog_table_row.iloc[0]['can_action_be_taken'])):
        new_state_last_action_taken = action_taken
        new_state_can_action_be_taken = False
        new_state_minutes_since_last_action = 0
    
    if ((dog_table_row.iloc[0]['last_action_taken'] == 'WALKING') &
        (dog_table_row.iloc[0]['minutes_since_last_action'] == WALKING_TIME)):
        new_state_fat, new_state_affection = update_walking(dog_table_row)
        new_state_last_action_taken = 'NO ACTION'
        new_state_can_action_be_taken = True
        new_state_minutes_since_last_action = 0        

    if ((dog_table_row.iloc[0]['last_action_taken'] == 'FEEDING') &
        (dog_table_row.iloc[0]['minutes_since_last_action'] == EATING_TIME)):
        new_state_food, new_state_fat = update_feeding(dog_table_row)
        new_state_last_action_taken = 'NO ACTION'
        new_state_can_action_be_taken = True
        new_state_minutes_since_last_action = 0

    if ((dog_table_row.iloc[0]['last_action_taken'] == 'PLAYING') &
        (dog_table_row.iloc[0]['minutes_since_last_action'] == PLAYING_TIME)):
        new_state_fat, new_state_affection = update_playing(dog_table_row)
        new_state_last_action_taken = 'NO ACTION'
        new_state_can_action_be_taken = True
        new_state_minutes_since_last_action = 0


    dog_table = dog_table.append({
        'dog_id': new_state_dog_id,
        'minute': new_state_minute,
        'happiness': new_state_happiness,
        'food': new_state_food,
        'fat': new_state_fat,
        'affection': new_state_affection,
        'alive': new_state_alive,
        'can_action_be_taken': new_state_can_action_be_taken, 
        'last_action_taken': new_state_last_action_taken,
        'minutes_since_last_action': new_state_minutes_since_last_action
        }, ignore_index=True)
    
    return dog_table

# Functions for RL

## Initialize dog

In [4]:
def init_dog_table():

    dog_table = pd.DataFrame(columns=[
        'dog_id', 'minute',
        'happiness', 'food', 'fat', 'affection',
        'alive', 'can_action_be_taken', 'last_action_taken', 'minutes_since_last_action'
    ])

    dog_table = dog_table.append({
        'dog_id': 1, 'minute': 0,
        'happiness': 0.50, 'food': 0.50, 'fat': 0, 'affection': 0.50,
        'alive': True, 'can_action_be_taken': True, 'last_action_taken': 'NO ACTION', 'minutes_since_last_action': 0
        }, ignore_index=True)
    
    return dog_table

## Choose action

In [5]:
epsilon = 0.3

def choose_action(state, Q, action_space):
    state_q = get_state_q(state)
    if np.random.uniform(0, 1) < epsilon:
        action = random.choices(range(len(action_space)), k = 1)[0]
    else:
        action = np.argmax(Q[state_q, :])
    return action

## Update Q function

In [6]:
def get_state_q(current_state):
    return int('{:02d}{:02d}{:02d}'.format(
        min(99,int(current_state.iloc[0]['food']*100)),
        min(99,int(current_state.iloc[0]['fat']*100)),
        min(99,int(current_state.iloc[0]['affection']*100))
    ))

In [7]:
# Function to learn the Q-value
alpha = 0.85
gamma = 0.95

def update(Q, state, state2, reward, action, action2):
    
    state_q = get_state_q(state)
    state2_q = get_state_q(state2)
    
    predict = Q[state_q, action]
    target = reward + gamma * Q[state2_q, action2]
    
    Q[state_q, action] = Q[state_q, action] + alpha * (target - predict)
    
    #print(Q[state_q, action])
    
    return Q

## Define reward

In [8]:
def define_reward(state, cnt_limit = 20):
    
    dog_table_next_hour = state.copy()
    cnt = 0
    
    while ((dog_table_next_hour.tail(1).iloc[0]['alive']) & (cnt < cnt_limit)):
        cnt += 1
        dog_table_next_hour = new_state_after_1_minute(dog_table_next_hour, action_taken='NO ACTION')

    average_happiness = dog_table_next_hour['happiness'].sum()/cnt_limit
    
    return average_happiness

# Q value function - initialisation

In [9]:
observation_space_n = 100*100*100
action_space = ['NO ACTION', 'WALKING', 'FEEDING', 'PLAYING']
action_space_n = len(action_space)

# This is where things are done

In [20]:
Q = np.zeros((observation_space_n, action_space_n))

In [21]:
all_overall_reward = []
all_overall_reward_t = []

In [35]:
max_steps = 1000
total_episodes = 100

In [40]:
# Starting the SARSA learning
for episode in trange(total_episodes):
    
    t = 0
    overall_reward = 0
    
    dog_table = init_dog_table()
    state1 = dog_table.tail(1)
    
    action1 = choose_action(
        state = state1,
        Q = Q,
        action_space = action_space)

    for i in range(max_steps):
        
        if not (state1.iloc[0]['alive']):
            break
        
        #Getting the next state
        dog_table = new_state_after_1_minute(
            dog_table,
            action_taken = action_space[action1])        
        state2 = dog_table.tail(1)
        
        reward = define_reward(state2)
        overall_reward += reward
        
        #Choosing the next action 
        action2 = choose_action(
            state = state2,
            Q = Q,
            action_space = action_space)
        
        #Learning the Q-value 
        Q = update(
            Q = Q,
            state = state1,
            state2 = state2,
            reward = reward,
            action = action1, 
            action2 = action2)
  
        state1 = state2 
        action1 = action2 
          
        #Updating the respective vaLues 
        t += 1
    
    all_overall_reward.append(overall_reward)
    all_overall_reward_t.append(overall_reward/t)
    
    print(episode, t, overall_reward, overall_reward/t)
    
    if len(all_overall_reward) % 10 == 0:
        
        plt.figure()
        plt.plot(range(len(all_overall_reward)), all_overall_reward)
        plt.show()

        plt.figure()
        plt.plot(range(len(all_overall_reward_t)), all_overall_reward_t)
        plt.show()

    
    #for j in range(Q.shape[1]):
    #    no_action = []
    #    for i in range(Q.shape[0]):
    #        no_action.append(Q[i,j])
    #    plt.plot(range(Q.shape[0]), no_action)
    #plt.show()


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(range(len(all_overall_reward)), all_overall_reward)
plt.show()

In [ ]:
for j in range(Q.shape[1]):
    no_action = []
    for i in range(Q.shape[0]):
        no_action.append(Q[i,j])
    plt.plot(range(Q.shape[0]), no_action)